# Day 5: Sunny with a Chance of Asteroids
You're starting to sweat as the ship makes its way toward Mercury. The Elves suggest that you get the air conditioner working by upgrading your ship computer to support the Thermal Environment Supervision Terminal.

The Thermal Environment Supervision Terminal (TEST) starts by running a diagnostic program (your puzzle input). The TEST diagnostic program will run on your existing Intcode computer after a few modifications:

First, you'll need to add two new instructions:

- Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
- Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.

Programs that use these instructions will come with documentation that explains what should be connected to the input and output. The program 3,0,4,0,99 outputs whatever it gets as input, then halts.

Second, you'll need to add support for parameter modes:

Each parameter of an instruction is handled based on its parameter mode. Right now, your ship computer already understands parameter mode 0, position mode, which causes the parameter to be interpreted as a position - if the parameter is 50, its value is the value stored at address 50 in memory. Until now, all parameters have been in position mode.

Now, your ship computer will also need to handle parameters in mode 1, immediate mode. In immediate mode, a parameter is interpreted as a value - if the parameter is 50, its value is simply 50.

Parameter modes are stored in the same value as the instruction's opcode. The opcode is a two-digit number based only on the ones and tens digit of the value, that is, the opcode is the rightmost two digits of the first value in an instruction. Parameter modes are single digits, one per parameter, read right-to-left from the opcode: the first parameter's mode is in the hundreds digit, the second parameter's mode is in the thousands digit, the third parameter's mode is in the ten-thousands digit, and so on. Any missing modes are 0.

For example, consider the program 1002,4,3,4,33.

The first instruction, 1002,4,3,4, is a multiply instruction - the rightmost two digits of the first value, 02, indicate opcode 2, multiplication. Then, going right to left, the parameter modes are 0 (hundreds digit), 1 (thousands digit), and 0 (ten-thousands digit, not present and therefore zero):

```
ABCDE
 1002

DE - two-digit opcode,      02 == opcode 2
 C - mode of 1st parameter,  0 == position mode
 B - mode of 2nd parameter,  1 == immediate mode
 A - mode of 3rd parameter,  0 == position mode,
                                  omitted due to being a leading zero
```

This instruction multiplies its first two parameters. The first parameter, 4 in position mode, works like it did before - its value is the value stored at address 4 (33). The second parameter, 3 in immediate mode, simply has value 3. The result of this operation, 33 * 3 = 99, is written according to the third parameter, 4 in position mode, which also works like it did before - 99 is written to address 4.

Parameters that an instruction writes to will never be in immediate mode.

Finally, some notes:
- It is important to remember that the instruction pointer should increase by the number of values in the instruction after the instruction finishes. Because of the new instructions, this amount is no longer always 4.
- Integers can be negative: 1101,100,-1,4,0 is a valid program (find 100 + -1, store the result in position 4).

The TEST diagnostic program will start by requesting from the user the ID of the system to test by running an input instruction - provide it 1, the ID for the ship's air conditioner unit.

It will then perform a series of diagnostic tests confirming that various parts of the Intcode computer, like parameter modes, function correctly. For each test, it will run an output instruction indicating how far the result of the test was from the expected value, where 0 means the test was successful. Non-zero outputs mean that a function is not working correctly; check the instructions that were run before the output instruction to see which one failed.

Finally, the program will output a diagnostic code and immediately halt. This final output isn't an error; an output followed immediately by a halt means the program finished. If all outputs were zero except the diagnostic code, the diagnostic program ran successfully.

After providing 1 to the only input instruction and passing all the tests, what diagnostic code does the program produce?

### the code from problem 2

In [61]:
class Computer(object):
    def __init__(self, opcodes, input_val, outputfn, verbose=True):
        self.opcodes = opcodes
        self.pos = 0
        self.input = input_val
        self.outputfn = outputfn
        
        self.verbose = verbose
    
    def log(self, msg):
        if self.verbose:
            print(msg)
    
    # opcode 1
    def add(self):
        addr1, addr2 = self.opcodes[self.pos + 1], self.opcodes[self.pos + 2]
        out = self.opcodes[self.pos + 3]
        val1, val2 = self.opcodes[addr1], self.opcodes[addr2]
        
        self.log(f'add: position {addr1} ({val1}) and {addr2} ({val2}) and storing in position {out}')
        
        self.opcodes[out]= val1 + val2
        self.pos += 4

    # opcode 2
    def mult(self):
        addr1, addr2 = self.opcodes[self.pos + 1], self.opcodes[self.pos + 2]
        out = self.opcodes[self.pos + 3]
        val1, val2 = self.opcodes[addr1], self.opcodes[addr2]
        
        self.log(f'mult: position {addr1} ({val1}) and {addr2} ({val2}) and storing in position {out}')
        
        self.opcodes[out]= val1 * val2
        self.pos += 4
    
    # opcode 3
    # Opcode 3 takes a single integer as input and saves it to the position 
    # given by its only parameter. For example, the instruction 3,50 would 
    # take an input value and store it at address 50.
    def save_input(self):
        dst = self.opcodes[self.pos+1]
        
        self.log(f'save_input: storing val={self.input} to dst={dst}')
        
        self.opcodes[dst] = self.input
        
        self.pos += 2
    
    # Opcode 4 outputs the value of its only parameter. For example, 
    # the instruction 4,50 would output the value at address 50.
    def send_output(self):
        addr = self.opcodes[self.pos+1]
        
        self.log(f'send_output: outputting addr={addr} val={self.opcodes[addr]}')
        
        self.outputfn(self.opcodes[addr])
        
        self.pos += 2

    # parse current instruction: get opcode, list of parameter modes
    # each opcode moves the instruction pointer a different length forward
    # parameter modes change how the params are interpreted:
    # position mode versus immediate mode
    
    # TODOs
    # - [x] add opcode 3
    # - [ ] add opcode 4
    # - [ ] add parameter modes
    # - [ ] parse instruction to get parameter modes
     
    def run(self):
        value = 0

        self.log(f'starting program: {self.opcodes}')

        while self.pos < len(self.opcodes):
            op = self.opcodes[self.pos]
            if op == 1:
                self.add()

            elif op == 2:
                self.mult()
                
            elif op == 3:
                self.save_input()
                
            elif op == 4:
                self.send_output()

            elif op == 99:
                self.log(f'halt, returning: {opcodes}')
                return self.opcodes

            else:
                raise ValueError(f'unknown opcode: {op}')

test cases from problem 2

In [62]:
cases = [
    # input, expected
    
    # 1,0,0,0,99 becomes 2,0,0,0,99 (1 + 1 = 2).
    ([1, 0, 0, 0, 99], [2, 0, 0, 0, 99]),

    # 2,3,0,3,99 becomes 2,3,0,6,99 (3 * 2 = 6).
    ([2,3,0,3,99], [2,3,0,6,99]),

    # 2,4,4,5,99,0 becomes 2,4,4,5,99,9801 (99 * 99 = 9801).
    ([2,4,4,5,99,0], [2,4,4,5,99,9801]),

    # 1,1,1,4,99,5,6,0,99 becomes 30,1,1,4,2,5,6,0,99.
    ([1,1,1,4,99,5,6,0,99], [30,1,1,4,2,5,6,0,99]),
]

for ops, expected in cases:
    assert Computer(ops, input_val=0, outputfn=print).run() == expected

starting program: [1, 0, 0, 0, 99]
add: position 0 (1) and 0 (1) and storing in position 0
halt, returning: [30, 1, 1, 4, 2, 5, 6, 0, 99]
starting program: [2, 3, 0, 3, 99]
mult: position 3 (3) and 0 (2) and storing in position 3
halt, returning: [30, 1, 1, 4, 2, 5, 6, 0, 99]
starting program: [2, 4, 4, 5, 99, 0]
mult: position 4 (99) and 4 (99) and storing in position 5
halt, returning: [30, 1, 1, 4, 2, 5, 6, 0, 99]
starting program: [1, 1, 1, 4, 99, 5, 6, 0, 99]
add: position 1 (1) and 1 (1) and storing in position 4
mult: position 5 (5) and 6 (6) and storing in position 0
halt, returning: [30, 1, 1, 4, 2, 5, 6, 0, 99]


In [63]:
my_input = "3,225,1,225,6,6,1100,1,238,225,104,0,1102,40,93,224,1001,224,-3720,224,4,224,102,8,223,223,101,3,224,224,1,224,223,223,1101,56,23,225,1102,64,78,225,1102,14,11,225,1101,84,27,225,1101,7,82,224,1001,224,-89,224,4,224,1002,223,8,223,1001,224,1,224,1,224,223,223,1,35,47,224,1001,224,-140,224,4,224,1002,223,8,223,101,5,224,224,1,224,223,223,1101,75,90,225,101,9,122,224,101,-72,224,224,4,224,1002,223,8,223,101,6,224,224,1,224,223,223,1102,36,63,225,1002,192,29,224,1001,224,-1218,224,4,224,1002,223,8,223,1001,224,7,224,1,223,224,223,102,31,218,224,101,-2046,224,224,4,224,102,8,223,223,101,4,224,224,1,224,223,223,1001,43,38,224,101,-52,224,224,4,224,1002,223,8,223,101,5,224,224,1,223,224,223,1102,33,42,225,2,95,40,224,101,-5850,224,224,4,224,1002,223,8,223,1001,224,7,224,1,224,223,223,1102,37,66,225,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,1007,226,677,224,1002,223,2,223,1005,224,329,1001,223,1,223,1007,226,226,224,1002,223,2,223,1006,224,344,101,1,223,223,1107,677,226,224,102,2,223,223,1006,224,359,1001,223,1,223,108,677,677,224,1002,223,2,223,1006,224,374,1001,223,1,223,107,677,677,224,1002,223,2,223,1005,224,389,101,1,223,223,8,677,677,224,1002,223,2,223,1005,224,404,1001,223,1,223,108,226,226,224,1002,223,2,223,1005,224,419,101,1,223,223,1008,677,677,224,1002,223,2,223,1005,224,434,101,1,223,223,1008,226,226,224,1002,223,2,223,1005,224,449,101,1,223,223,7,677,226,224,1002,223,2,223,1006,224,464,1001,223,1,223,7,226,226,224,1002,223,2,223,1005,224,479,1001,223,1,223,1007,677,677,224,102,2,223,223,1005,224,494,101,1,223,223,1108,677,226,224,102,2,223,223,1006,224,509,1001,223,1,223,8,677,226,224,102,2,223,223,1005,224,524,1001,223,1,223,1107,226,226,224,102,2,223,223,1006,224,539,1001,223,1,223,1008,226,677,224,1002,223,2,223,1006,224,554,1001,223,1,223,1107,226,677,224,1002,223,2,223,1006,224,569,1001,223,1,223,1108,677,677,224,102,2,223,223,1005,224,584,101,1,223,223,7,226,677,224,102,2,223,223,1006,224,599,1001,223,1,223,1108,226,677,224,102,2,223,223,1006,224,614,101,1,223,223,107,226,677,224,1002,223,2,223,1005,224,629,101,1,223,223,108,226,677,224,1002,223,2,223,1005,224,644,101,1,223,223,8,226,677,224,1002,223,2,223,1005,224,659,1001,223,1,223,107,226,226,224,1002,223,2,223,1006,224,674,101,1,223,223,4,223,99,226"

my_input = list(map(int, my_input.split(",")))

In [64]:
Computer(my_input, input_val=1, outputfn=print).run()

starting program: [3, 225, 1, 225, 6, 6, 1100, 1, 238, 225, 104, 0, 1102, 40, 93, 224, 1001, 224, -3720, 224, 4, 224, 102, 8, 223, 223, 101, 3, 224, 224, 1, 224, 223, 223, 1101, 56, 23, 225, 1102, 64, 78, 225, 1102, 14, 11, 225, 1101, 84, 27, 225, 1101, 7, 82, 224, 1001, 224, -89, 224, 4, 224, 1002, 223, 8, 223, 1001, 224, 1, 224, 1, 224, 223, 223, 1, 35, 47, 224, 1001, 224, -140, 224, 4, 224, 1002, 223, 8, 223, 101, 5, 224, 224, 1, 224, 223, 223, 1101, 75, 90, 225, 101, 9, 122, 224, 101, -72, 224, 224, 4, 224, 1002, 223, 8, 223, 101, 6, 224, 224, 1, 224, 223, 223, 1102, 36, 63, 225, 1002, 192, 29, 224, 1001, 224, -1218, 224, 4, 224, 1002, 223, 8, 223, 1001, 224, 7, 224, 1, 223, 224, 223, 102, 31, 218, 224, 101, -2046, 224, 224, 4, 224, 102, 8, 223, 223, 101, 4, 224, 224, 1, 224, 223, 223, 1001, 43, 38, 224, 101, -52, 224, 224, 4, 224, 1002, 223, 8, 223, 101, 5, 224, 224, 1, 223, 224, 223, 1102, 33, 42, 225, 2, 95, 40, 224, 101, -5850, 224, 224, 4, 224, 1002, 223, 8, 223, 1001, 224, 7,

ValueError: unknown opcode: 1101

In [59]:
len(my_input)

678